<a href="https://colab.research.google.com/github/vladana-perlic/M1_Memoire_de_recherche/blob/main/V2_Camembert_masquage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.0 MB/s eta 0:00:00


In [ ]:
from transformers import CamembertTokenizer
from transformers import CamembertForMaskedLM
import torch

In [ ]:
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")
model.eval()

def fill_mask(masked_input, model, tokenizer, topk=5):
    # Adapted from https://github.com/pytorch/fairseq/blob/master/fairseq/models/roberta/hub_interface.py
    assert masked_input.count("<mask>") == 1
    input_ids = torch.tensor(tokenizer.encode(masked_input, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    logits = model(input_ids)[0]  # The last hidden-state is the first element of the output tuple
    masked_index = (input_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()
    logits = logits[0, masked_index, :]
    prob = logits.softmax(dim=0)
    values, indices = prob.topk(k=topk, dim=0)
    topk_predicted_token_bpe = " ".join(
        [tokenizer.convert_ids_to_tokens(indices[i].item()) for i in range(len(indices))]
    )
    masked_token = tokenizer.mask_token
    topk_filled_outputs = []
    for index, predicted_token_bpe in enumerate(topk_predicted_token_bpe.split(" ")):
        predicted_token = predicted_token_bpe.replace("\u2581", " ")
        if " {0}".format(masked_token) in masked_input:
            topk_filled_outputs.append(
                (
                    masked_input.replace(" {0}".format(masked_token), predicted_token),
                    values[index].item(),
                    predicted_token,
                )
            )
        else:
            topk_filled_outputs.append(
                (masked_input.replace(masked_token, predicted_token), values[index].item(), predicted_token,)
            )
    return topk_filled_outputs

## Code à modifier qu'à partir d'ici

# **VERSION 2, EXPORTER LES VALEURS DANS UN FICHIER EXCEL**

In [ ]:
import pandas as pd
from google.colab import files

# importer le fichier Excel
url = 'https://docs.google.com/spreadsheets/d/1XwPNkpb2YDUojbYYbCE_F4Un-40RyfgC/export?format=xlsx'
df = pd.read_excel(url)

cam_vocab = tokenizer.get_vocab()
# remplacer le mot par "<mask>" en fonction du statut
for i, row in df.iterrows():
    if row["mot ajouté --> supprimé --> modifié"] in cam_vocab:
        if row['classification'] == 'ajouté':
            masked_input = row["requête reformulée"].replace(row["mot ajouté --> supprimé --> modifié"], "<mask>")
            topk = 32005
            results = fill_mask(masked_input, model, tokenizer, topk=topk)
            for index, result in enumerate(results):
                if row["mot ajouté --> supprimé --> modifié"] in result[2]:
                    df.loc[i, 'topk'] = index
                    break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'] == 'supprimé':
            masked_input = row["requête originale"].replace(row["mot ajouté --> supprimé --> modifié"], "<mask>")
            topk = 32005
            results = fill_mask(masked_input, model, tokenizer, topk=topk)
            for index, result in enumerate(results):
                if row["mot ajouté --> supprimé --> modifié"] in result[2]:
                    df.loc[i, 'topk'] = index
                    break
                else:
                    df.loc[i, 'topk'] = "NA"
        elif row['classification'] == 'modifié':
            mot2 = row["mot ajouté --> supprimé --> modifié"].split(" --> ")[1]
            if mot2 in cam_vocab:
                masked_input = row["requête originale"].replace(row["mot ajouté --> supprimé --> modifié"], "<mask>")
                topk = 32005
                results = fill_mask(masked_input, model, tokenizer, topk=topk)
                for index, result in enumerate(results):
                    if mot2 in result[2]:
                        df.loc[i, 'topk'] = index
                        break
                    else:
                        df.loc[i, 'topk'] = "NA"

# exporter les données mises à jour dans un fichier Excel local
filename = "output.xlsx"
df.to_excel(filename, index=False, engine='openpyxl')

# télécharger le fichier Excel
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **VERSION 1, EXPORTER LES VALEURS DANS UN FICHIER EXCEL**

In [ ]:
import pandas as pd
from google.colab import files

# importer le fichier Excel
url = 'https://docs.google.com/spreadsheets/d/1XwPNkpb2YDUojbYYbCE_F4Un-40RyfgC/export?format=xlsx'
df = pd.read_excel(url)

# remplacer le mot par "<mask>" en fonction du statut
for i, row in df.iterrows():
    if row['classification'] in 'ajouté' and row['langue'] in 'fr':
        masked_input = row["requête reformulée"].replace(row["mot ajouté --> supprimé --> modifié"], "<mask>")
        found = False
        topk = 100
        while not found and topk <= 30000:
            results = fill_mask(masked_input, model, tokenizer, topk=topk)
            for result in results:
                if row["mot ajouté --> supprimé --> modifié"] in result[2]:
                    found = True
                    df.loc[i, 'topk'] = topk
                    break
            topk *= 2 # augmenter topk de façon exponentielle
        if not found:
            df.loc[i, 'topk'] = "NA"
    elif row['classification'] in 'supprimé' and row['langue'] in 'fr':
        masked_input = row["requête originale"].replace(row["mot ajouté --> supprimé --> modifié"], "<mask>")
        found = False
        topk = 100
        while not found and topk <= 30000:
            results = fill_mask(masked_input, model, tokenizer, topk=topk)
            for result in results:
                if row["mot ajouté --> supprimé --> modifié"] in result[2]:
                    found = True
                    df.loc[i, 'topk'] = topk
                    break
            topk *= 2 # augmenter topk de façon exponentielle
        if not found:
            df.loc[i, 'topk'] = "NA"

# exporter les données mises à jour dans un fichier Excel local
filename = "output.xlsx"
df.to_excel(filename, index=False, engine='openpyxl')

# télécharger le fichier Excel
files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# EXEMPLE ORIGINAL :

In [ ]:
#utiliser mask pour remplacer le mot cherché (donc ajouté ou supprimé)

masked_input = "Le Camembert est <mask> :)"
[i[2] for i in fill_mask(masked_input, model, tokenizer, topk=50)]
# print(fill_mask(masked_input, model, tokenizer, topk=50))

In [ ]:
#permet de voir comment sont tokenisés les mots, si +1 token alors le mot est absent dans le voc
sentences = ["bruit"]
for sent in sentences:
    print('Original: ', sent)
    print('Tokenized Camembert: ', tokenizer.tokenize(sent))

Original:  bruit
Tokenized Camembert:  ['▁bruit']


In [ ]:
#teste directement pour un mot sa présence dans le voc, si mot pas présent message "false" sinon "true"
cam_vocab = tokenizer.get_vocab()
if "chat" not in cam_vocab:
  print("false")
else :
  print("true")


# for v in ["insomniaque"]:
#     print(cam_vocab[v],end = " ")

true
